In [1]:
def MC(beta):
    ans_MC = MCA(beta, run_limit, cost_or, cost_rs, cost_sd, cost_sr, cost_od,
        parameter_p, parameter_c, 
        O, R, S, D, RS, 
        num_driver_od, num_shipper_rs, fixed_cost_do_shipper_rs, fixed_cost_dont_shipper_rs)[0]
    return ans_MC

def calculate_grad_MC(beta):
    ans_nabla_MC = MCA(beta, run_limit, cost_or, cost_rs, cost_sd, cost_sr, cost_od,
        parameter_p, parameter_c, 
        O, R, S, D, RS, 
        num_driver_od, num_shipper_rs, fixed_cost_do_shipper_rs, fixed_cost_dont_shipper_rs)[1]
    return ans_nabla_MC

def bector_of_MC(beta, step_size, RS, grad_MC):
    Bector_of_MC = {}
    for rs in RS:
        Bector_of_MC[rs] = beta.get(rs) + step_size * grad_MC.get(rs)   # from def nablaMC
    return Bector_of_MC

def calc_diff(rs, v_after, v_before):
    difference = np.zeros(len(RS))
    i = 0
    for rs in RS:
        difference[i] = v_after.get(rs) - v_before.get(rs)
        i += 1
    return difference



def fista(ipsilon, eta, stepsize, RS):
    max_inner_iter = 100000
    max_outer_iter = 10000
    # beta = np.array([5.0, 0.0, -3.0])  # 初期値

    beta = {}
    for rs in RS:
        beta[rs] = 1   # 初期値（=1）
    
    v_after = copy.deepcopy(beta)
    t = 1
    iota = 0
    min_step_size = 1e-20  # ステップサイズの最小値

    for k in range(max_outer_iter):

        # あらかじめ grad_MC を計算
        grad_MC = calculate_grad_MC(beta)
        
        iota = 0  # 内部ループのカウンタをリセット
        while iota < max_inner_iter:
            step_size = 1 / (stepsize * eta**iota)
            if step_size < min_step_size:
                print("Warning: Step size became too small.")
                return beta  # 現時点の推定値を返す
            
            F = MC( bector_of_MC(beta, step_size, RS, grad_MC) )   # from def bector_of_MC
            Q = MC(beta) - (step_size / 2) * np.linalg.norm( list(grad_MC.values()) )**2
            if F >= Q:
                break
            iota += 1

        # 内部ループが最大反復数に達した場合の処理
        if iota == max_inner_iter:
            print("Warning: Inner loop reached maximum iterations.")
            return beta

        # ステップサイズの調整
        stepsize = stepsize / eta

        # FISTAの更新
        v_before = copy.deepcopy(v_after)
        v_after.clear()
        for rs in RS:
            v_after[rs] = beta.get(rs) + 1 / stepsize * grad_MC.get(rs)
        
        # 判定用の grad_MC を計算
        judg_grad_MC = calculate_grad_MC(v_after)   # from def calculate_grad_MC
        
        # 収束判定
        if np.linalg.norm( list(judg_grad_MC.values()) ) < ipsilon:
            print(f"Converged after {k + 1} outer iterations.")
            return v_after

        # 加速パラメータの更新
        if np.dot(list(judg_grad_MC.values()), calc_diff(rs, v_after, v_before)) < 0:
            t_before = 1
        else:
            t_before = t
            
        t = (1 + (1 + 4 * t_before**2)**0.5) / 2

        beta.clear()
        for rs in RS:
            beta[rs] = v_after.get(rs) + ( t_before - 1 ) / t * ( v_after.get(rs) - v_before.get(rs) )
        

    # 外部ループが最大反復数に達した場合
    print(f"Warning: Outer loop reached maximum iterations without full convergence.")
    return beta

# テスト用
# def MC(x):
#     """
#     多次元二次関数: (1/2) * ||x - c||^2
#     x: np.array, 入力ベクトル
#     """
#     c = np.array([3.0, -1.0, 2.0])  # 最適解の座標
#     return 0.5 * np.linalg.norm(x - c)**2

# def nablaMC(x):
#     """
#     多次元二次関数の勾配: x - c
#     x: np.array, 入力ベクトル
#     """
#     c = np.array([3.0, -1.0, 2.0])  # 最適解の座標
#     return x - c

# def MC(beta):
#     return sum(100 * (beta[i+1] - beta[i]**2)**2 + (1 - beta[i])**2 for i in range(len(beta) - 1))

# def nablaMC(beta):
#     grad = np.zeros_like(beta)
#     for i in range(len(beta) - 1):
#         grad[i] += -400 * beta[i] * (beta[i+1] - beta[i]**2) - 2 * (1 - beta[i])
#         grad[i+1] += 200 * (beta[i+1] - beta[i]**2)
#     return grad

# # 実行
# result = fista(1e-4, 1.05, 2.0)
# print("最適解:", result)
